In [1]:
from dask import persist
from dask.distributed import Client, progress
import dask.dataframe as dd
from dask_ml.model_selection import train_test_split
import dask
import dask_xgboost

/home/samir/anaconda3/envs/py36/lib/python3.6/site-packages/dask/array/random.py:27: FutureWarning: dask.array.random.doc_wraps is deprecated and will be removed in a future version
  FutureWarning,


In [2]:
from src.utils import download_and_extract, handle_data_path, list_of_df, concatenate_to_df_from_np

In [3]:
URL = "https://storage.googleapis.com/dask-tutorial-data/nycflights.tar.gz"
DATA_PATH = 'data'

In [4]:
tar_path, csv_path = handle_data_path('data', URL, )
download_and_extract('data', URL, tar_path)

True

In [5]:
client = Client(threads_per_worker=4,
                n_workers=1, memory_limit='2GB')
client.restart()

Client Scheduler: tcp://127.0.0.1:33739 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


In [6]:
cols = ['Year', 'Month', 'DayOfWeek', 'Distance',
        'DepDelay', 'CRSDepTime', 'UniqueCarrier', 'Origin', 'Dest']

In [7]:
df_list = list_of_df(csv_path, 10000)
columns = df_list[0].columns
df = concatenate_to_df_from_np(df_list)
df = df[cols]

In [8]:
print(df.shape)
print(df.dropna().shape)
df = df.dropna()

(100000, 9)
(94277, 9)


In [9]:
# Subset of the columns to use

# Create the dataframe
df = dd.from_pandas(df, chunksize=100)

delay = df['DepDelay']
del df['DepDelay']  # Remove delay information from training dataframe

df, delay = persist(df, delay)  # Ask Dask to start work on these in the background
progress(df, delay)

VBox()

In [10]:
delay.head()

0     0.0
0     1.0
0    11.0
0    -1.0
0     0.0
Name: DepDelay, dtype: float64

In [11]:
df = dd.get_dummies(df.categorize()).persist()

In [12]:
X = df.to_dask_array(lengths=True)
y = delay.to_dask_array(lengths=True)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

In [14]:
params = {'objective': 'reg:squarederror',
          'max_depth': 4, 'eta': 0.01, 'subsample': 0.5,
          'min_child_weight': 0.5}

bst = dask_xgboost.train(client, params, X_train, y_train, num_boost_round=10)

In [15]:
y_hat = dask_xgboost.predict(client, bst, X_test).persist()
y_hat

,Array,Chunk
Bytes,76.96 kB,88 B
Shape,"(19239,)","(22,)"
Count,943 Tasks,943 Chunks
Type,float32,numpy.ndarray


In [16]:
y_hat

,Array,Chunk
Bytes,76.96 kB,88 B
Shape,"(19239,)","(22,)"
Count,943 Tasks,943 Chunks
Type,float32,numpy.ndarray


In [17]:
from sklearn.metrics import mean_squared_error

y_test, y_hat = dask.compute(y_test, y_hat)
mean_squared_error(y_test, y_hat)

1124.7423765047777

In [19]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

param_grid = {"alpha": [0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0],}

grid_search = GridSearchCV(Ridge(random_state=0),
                           param_grid=param_grid,
                           return_train_score=False,
                           iid=True,
                           cv=3,
                           n_jobs=-1)

In [ ]:
import joblib

with joblib.parallel_backend('dask'):
    grid_search.fit(X_train, y_train)

In [16]:
y_hat_train = grid_search.best_estimator_.predict(X_train)
y_hat_test = grid_search.best_estimator_.predict(X_test)

In [17]:
print(mean_squared_error(y_train, y_hat_train))
print(mean_squared_error(y_test, y_hat_test))

NameError: name 'mean_squared_error' is not defined